In [ ]:
try:
    from ase import Atoms
    from ase.calculators.lj import LennardJones
    print("successfully imported ase")
except ImportError:
    !! pip install ase --user --upgrade
    print("completed installing ASE")

In [ ]:
try:
    import torch
    print("successfully imported torch")
except ImportError:
    !! pip install torch --user --upgrade
    print("completed installing torch")

In [ ]:
try:
    import dscribe
    print("successfully imported dscribe")
except ImportError:
    !! pip install dscribe --user --upgrade
    print("completed installing dscribe")

In [ ]:
try:
    import numpy as np
    print("successfully imported numpy")
except ImportError:
    !! pip install numpy --user --upgrade
    print("completed installing numpy")

In [ ]:
try:
    import aenet_gpr
    print("successfully imported aenet_gpr")
except ImportError:
    !! pip install aenet_gpr --user --upgrade
    print("completed installing aenet_gpr")

# 1. Prepare training/test data

In [ ]:
!! unzip ../aenet_example/3_Li-EC/train_set.zip -d ./3_Li-EC/

In [ ]:
!! unzip ../aenet_example/3_Li-EC/test_set.zip -d ./3_Li-EC/

# 2. Train–Test–Augmentation execute

In [1]:
! cat ./3_Li-EC/train.in

# File path
Train_file ./train_set/file_*.xsf
Test_file ./test_set/file_*.xsf

# File format (default: xsf)
File_format xsf

# Descriptor (default: cartesian coordinates)
Descriptor cart

# Uncertainty estimation (default: True)
Get_variance True

# Kernel parameter (default: Squared exponential)
scale 3.0
weight 1.0

# Chunking to reduce memory usage (default: batch, 25)
data_process batch
batch_size 25

# Flags for xsf file writing (default: False)
Train_write False
Test_write False
# Additional_write True

# Data augmentation parameter (default: 0.055, 25)
Disp_length 0.05
Num_copy 5


In [ ]:
! python aenet_GPR/aenet_gpr.py ./3_Li-EC/train.in > ./3_Li-EC/train.out